In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import datetime
import yfinance as yf
import math
import matplotlib.pyplot as plt



In [2]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)
    

In [3]:
def backtest_supertrend(df, ticker, investment=10000):
    df['Date'] = df.index
    is_uptrend = df['Supertrend']
    close = df['Close']
    date = df['Date']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 50
    share = 0
    entry = []
    exit = []
    tradeCount=0
    buyDate = ''
    sellDate = ''
    buyPrice = 0
    sellPrice = 0
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = math.floor(equity / close[i] )
            equity -= share * close[i]
            buyDate = date[i]
            entry.append((i, close[i], buyDate))
            in_position = True
            tradeCount += 1
            
            #print(f'Buy {share} shares at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            sellDate = date[i]
            exit.append((i, close[i], sellDate, equity))
            in_position = False
            
            #print(f'Sell at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'{ticker} :\t Earning from investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, roi, tradeCount

In [4]:
def backtest_supertrend_profit(df, ticker, investment=10000):
    df['Date'] = df.index
    is_uptrend = df['Supertrend']
    close = df['Close']
    date = df['Date']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 50
    share = 0
    entry = []
    exit = []
    tradeCount=0
    buyDate = ''
    sellDate = ''
    buyPrice = 0
    sellPrice = 0
    targetPrice=0
    stopLoss=0
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = math.floor(equity / close[i] )
            equity -= share * close[i]
            buyDate = date[i]
            entry.append((i, close[i], buyDate))
            in_position = True
            targetPrice = close[i] * 1.2
            stopLoss = close[i] * 0.95
            tradeCount += 1
            
            #print(f'Buy {share} shares at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and (not is_uptrend[i] or close[i] > targetPrice or close[i] < stopLoss):
            equity += share * close[i] - commission
            sellDate = date[i]
            exit.append((i, close[i], sellDate, equity))
            in_position = False
            
            #print(f'Sell at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning / investment * 100, 2)
    print(f'{ticker} :\t Earning from investing {investment} is ${round(earning,2)} \t (ROI = {roi}%) \t Trades={tradeCount}')
    return entry, exit, earning, roi, tradeCount

In [5]:
def find_optimal_parameter(df, ticker):
    # predefine several parameter sets
    atr_period = [1,2,3,4,5,6,7, 8, 9, 10]
    atr_multiplier = [1.0, 1.5, 2.0, 2.5,2.75, 3.0,4,4.5,5]
    roi_list = []
    
    # for each period and multiplier, perform backtest
    for period, multiplier in [(x,y) for x in atr_period for y in atr_multiplier]:
        new_df = df
        supertrend = Supertrend(df, period, multiplier)
        new_df = df.join(supertrend)
        new_df = new_df[period:]
        entry, exit, roi, tradeCount = backtest_supertrend(new_df,ticker, 100000)
        roi_list.append((period, multiplier, roi, tradeCount))
    
    print(pd.DataFrame(roi_list, columns=['ATR_period','Multiplier','ROI','TradeCount']).to_string())
    
    # return the best parameter set
    return max(roi_list, key=lambda x:x[2])

In [6]:
def downloadIndividualSymbol():
    symbol = 'JINDALSTEL.NS' 
    df = yf.download(symbol, start='2019-01-01') 
    df

In [7]:
def sortSecond(val):
    return val[2] 

In [8]:
def NSEData(tickerList, startDate, endDate, interval='1D'):
    

    if(interval=='1h'):
        startDate= dt.today() - datetime.timedelta(days=70)
        endDate=dt.today()

    #Fetch NSE Ticker Data
    result=[] 
    for ticker in tickerList: 
        data = []
        symbol = ticker + ".NS" 
        df = yf.download(symbol, start=startDate, end=endDate, interval=interval) 
        data.append(symbol)
        data.append(df)
        result.append(data)

    return result    

In [9]:
def STBackTest(dataframes,atr_period,atr_multiplier):
    ROI =[]
    Earnings = []
    investment = []
    totalTrades = []
    for data in dataframes:
        df=data[1]
        ticker=data[0]
        supertrend = Supertrend(df, atr_period, atr_multiplier)
        df = df.join(supertrend)
        entry, exit, earning, roi, tradeCount=backtest_supertrend(df,ticker, 10000)
        Earnings.append(earning)
        ROI.append(roi)
        investment.append(10000)
        totalTrades.append(tradeCount)
    return sum(Earnings), sum(investment), sum(totalTrades)

In [10]:
def find_optimal_parameter_for_NSE(dataframes):
    # predefine several parameter sets
    atr_period = [1,2,3,4,5,6,7, 8, 9, 10]
    atr_multiplier = [1.0, 1.5, 2.0, 2.5,2.75, 3.0,4,4.5,5]
    roi_list = []
    
    # for each period and multiplier, perform backtest
    for period, multiplier in [(x,y) for x in atr_period for y in atr_multiplier]:
        returns, investment, totalTrades= STBackTest(dataframes,period, multiplier)
        roi = round(returns/investment*100,2)
        print(f'Investments : {investment}, Returns : {returns}, ROI : {roi}, Period : {period}, Multiplier : {multiplier}, TotalTrades : {totalTrades}')
        #print(pd.DataFrame(roi_list, columns=['ATR_period','Multiplier','ROI']))
    
        roi_list.append((period, multiplier, roi, investment, returns, totalTrades))
    
    # return the best parameter set
    return roi_list

In [11]:
# visualization
def visualization(df):
    plt.plot(df['Adj Close'], label='Close Price')
    plt.plot(df['Final Lowerband'], 'g', label = 'Final Lowerband')
    plt.plot(df['Final Upperband'], 'r', label = 'Final Upperband')
    plt.show()

In [12]:
# get the full stock list of S&P 500

def SNPStockList():
    atr_period=4
    atr_multiplier=4
    payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    stock_list = payload[0]['Symbol'].values.tolist()
    supertrend_stocks = []
    # loop through each symbol
    for symbol in stock_list:
        df = yf.download(symbol, start='2010-01-01', threads= False)
        if len(df) == 0: continue
        supertrend = Supertrend(df, atr_period, atr_multiplier)
        if not supertrend['Supertrend'][-2] and supertrend['Supertrend'][-1]:
            supertrend_stocks.append(symbol)

    for s in supertrend_stocks :
        print(s, end = ', ')

In [13]:
def STStockList(stock_list):
    atr_period=4
    atr_multiplier=4
    supertrend_stocks = []
    # loop through each symbol
    
    for ticker in tickerList: 
        data = []
        symbol = ticker + ".NS" 
        df = yf.download(symbol, start='2020-01-01') 
        if len(df) == 0: continue
        supertrend = Supertrend(df, atr_period, atr_multiplier)
        if not supertrend['Supertrend'][-2] and supertrend['Supertrend'][-1]:
            supertrend_stocks.append(symbol)
    
    for s in supertrend_stocks :
        print(s, end = ', ')

# To get the S7P trades and identify the stock alert

In [ ]:
# To get the S7P trades and identify the stock alert
SNPStockList()

# To Fetch NSE Data

In [ ]:
# input Variables
startDate='2021-01-01'
endDate='2021-12-31'
#Load NSE Ticker from WIKI
#tickerList = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1].Symbol.to_list()

#BlueChip Trend from TV is giving more returns
tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']

nseDF = NSEData(tickerList,startDate,endDate)


# Get today's ST Signal

In [ ]:
# Get today's ST Signal
tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']
STStockList(tickerList)

# find_optimal_parameter_for_NSE 

In [55]:
#find_optimal_parameter_for_NSE 

#Test below thoroughly
# 1) 3;5;213.75;796
# 2) 4;4;222.96;1011 --most promissing
# 3) 7;2.5;225.96;1914
# 4) 10;2.5;222.31;1927
# 5) 5;2.5;232.38;1928




roi_list = find_optimal_parameter_for_NSE(nseDF)
roi_list.sort(key = sortSecond,reverse=True) 
roiDF = pd.DataFrame(roi_list,columns =['period','multiplier','roi','investment','finalReturn','totalTrades'])


optimal_param = max(roi_list, key=lambda x:x[2])
print(f'Best parameter set: ATR Period={optimal_param[0]}, Multiplier={optimal_param[1]}, ROI={optimal_param[2]}')
#print(roiDF.to_string())
roiDF.to_csv('result/NSE_SUPERTREND_BACKTEST.csv')

ADANIPORTS.NS :	 Earning from investing 10000 is $-4269.75 	 (ROI = -42.7%) 	 Trades=107
ASIANPAINT.NS :	 Earning from investing 10000 is $-2283.75 	 (ROI = -22.84%) 	 Trades=109
AXISBANK.NS :	 Earning from investing 10000 is $-6444.7 	 (ROI = -64.45%) 	 Trades=117
BAJAJ-AUTO.NS :	 Earning from investing 10000 is $-3612.9 	 (ROI = -36.13%) 	 Trades=103
BAJFINANCE.NS :	 Earning from investing 10000 is $75049.13 	 (ROI = 750.49%) 	 Trades=101
BAJAJFINSV.NS :	 Earning from investing 10000 is $15883.9 	 (ROI = 158.84%) 	 Trades=109
BHARTIARTL.NS :	 Earning from investing 10000 is $-10096.36 	 (ROI = -100.96%) 	 Trades=118
BPCL.NS :	 Earning from investing 10000 is $449.73 	 (ROI = 4.5%) 	 Trades=106
BRITANNIA.NS :	 Earning from investing 10000 is $11121.18 	 (ROI = 111.21%) 	 Trades=99
CIPLA.NS :	 Earning from investing 10000 is $-6683.0 	 (ROI = -66.83%) 	 Trades=98
COALINDIA.NS :	 Earning from investing 10000 is $-8229.75 	 (ROI = -82.3%) 	 Trades=112
DIVISLAB.NS :	 Earning from investin

SBILIFE.NS :	 Earning from investing 10000 is $-2238.15 	 (ROI = -22.38%) 	 Trades=28
SHREECEM.NS :	 Earning from investing 10000 is $-2376.2 	 (ROI = -23.76%) 	 Trades=43
SUNPHARMA.NS :	 Earning from investing 10000 is $-6840.8 	 (ROI = -68.41%) 	 Trades=48
TATAMOTORS.NS :	 Earning from investing 10000 is $32347.3 	 (ROI = 323.47%) 	 Trades=40
TATASTEEL.NS :	 Earning from investing 10000 is $13703.66 	 (ROI = 137.04%) 	 Trades=44
TCS.NS :	 Earning from investing 10000 is $656.75 	 (ROI = 6.57%) 	 Trades=48
TATACONSUM.NS :	 Earning from investing 10000 is $17944.85 	 (ROI = 179.45%) 	 Trades=35
TECHM.NS :	 Earning from investing 10000 is $518.01 	 (ROI = 5.18%) 	 Trades=44
TITAN.NS :	 Earning from investing 10000 is $18304.65 	 (ROI = 183.05%) 	 Trades=37
ULTRACEMCO.NS :	 Earning from investing 10000 is $4283.15 	 (ROI = 42.83%) 	 Trades=43
UPL.NS :	 Earning from investing 10000 is $353.85 	 (ROI = 3.54%) 	 Trades=44
WIPRO.NS :	 Earning from investing 10000 is $5345.3 	 (ROI = 53.45%) 

INFY.NS :	 Earning from investing 10000 is $9058.9 	 (ROI = 90.59%) 	 Trades=20
IOC.NS :	 Earning from investing 10000 is $-1026.01 	 (ROI = -10.26%) 	 Trades=23
ITC.NS :	 Earning from investing 10000 is $-4979.4 	 (ROI = -49.79%) 	 Trades=24
JSWSTEEL.NS :	 Earning from investing 10000 is $11325.31 	 (ROI = 113.25%) 	 Trades=21
KOTAKBANK.NS :	 Earning from investing 10000 is $6479.23 	 (ROI = 64.79%) 	 Trades=19
LT.NS :	 Earning from investing 10000 is $6061.46 	 (ROI = 60.61%) 	 Trades=20
M&M.NS :	 Earning from investing 10000 is $-3749.93 	 (ROI = -37.5%) 	 Trades=26
MARUTI.NS :	 Earning from investing 10000 is $223.0 	 (ROI = 2.23%) 	 Trades=26
NESTLEIND.NS :	 Earning from investing 10000 is $5002.9 	 (ROI = 50.03%) 	 Trades=20
NTPC.NS :	 Earning from investing 10000 is $-1323.0 	 (ROI = -13.23%) 	 Trades=19
ONGC.NS :	 Earning from investing 10000 is $-1898.23 	 (ROI = -18.98%) 	 Trades=20
POWERGRID.NS :	 Earning from investing 10000 is $-2645.08 	 (ROI = -26.45%) 	 Trades=22
RELIAN

COALINDIA.NS :	 Earning from investing 10000 is $-5786.75 	 (ROI = -57.87%) 	 Trades=46
DIVISLAB.NS :	 Earning from investing 10000 is $14765.32 	 (ROI = 147.65%) 	 Trades=37
DRREDDY.NS :	 Earning from investing 10000 is $-505.4 	 (ROI = -5.05%) 	 Trades=47
EICHERMOT.NS :	 Earning from investing 10000 is $-3494.9 	 (ROI = -34.95%) 	 Trades=54
GRASIM.NS :	 Earning from investing 10000 is $8131.61 	 (ROI = 81.32%) 	 Trades=41
HCLTECH.NS :	 Earning from investing 10000 is $2922.81 	 (ROI = 29.23%) 	 Trades=40
HDFC.NS :	 Earning from investing 10000 is $5936.2 	 (ROI = 59.36%) 	 Trades=41
HDFCBANK.NS :	 Earning from investing 10000 is $5811.45 	 (ROI = 58.11%) 	 Trades=40
HDFCLIFE.NS :	 Earning from investing 10000 is $1309.75 	 (ROI = 13.1%) 	 Trades=24
HEROMOTOCO.NS :	 Earning from investing 10000 is $-2615.25 	 (ROI = -26.15%) 	 Trades=45
HINDALCO.NS :	 Earning from investing 10000 is $13478.95 	 (ROI = 134.79%) 	 Trades=42
HINDUNILVR.NS :	 Earning from investing 10000 is $14487.95 	 (R

WIPRO.NS :	 Earning from investing 10000 is $6561.06 	 (ROI = 65.61%) 	 Trades=27
Investments : 500000, Returns : 468119.9909133911, ROI : 93.62, Period : 2, Multiplier : 3.0, TotalTrades : 1236
ADANIPORTS.NS :	 Earning from investing 10000 is $562.1 	 (ROI = 5.62%) 	 Trades=20
ASIANPAINT.NS :	 Earning from investing 10000 is $10159.7 	 (ROI = 101.6%) 	 Trades=18
AXISBANK.NS :	 Earning from investing 10000 is $1844.8 	 (ROI = 18.45%) 	 Trades=16
BAJAJ-AUTO.NS :	 Earning from investing 10000 is $-620.4 	 (ROI = -6.2%) 	 Trades=18
BAJFINANCE.NS :	 Earning from investing 10000 is $117599.86 	 (ROI = 1176.0%) 	 Trades=16
BAJAJFINSV.NS :	 Earning from investing 10000 is $99956.4 	 (ROI = 999.56%) 	 Trades=14
BHARTIARTL.NS :	 Earning from investing 10000 is $706.8 	 (ROI = 7.07%) 	 Trades=17
BPCL.NS :	 Earning from investing 10000 is $-1198.38 	 (ROI = -11.98%) 	 Trades=21
BRITANNIA.NS :	 Earning from investing 10000 is $9796.1 	 (ROI = 97.96%) 	 Trades=18
CIPLA.NS :	 Earning from investing 

...

WIPRO.NS :	 Earning from investing 10000 is $7964.33 	 (ROI = 79.64%) 	 Trades=39
Investments : 500000, Returns : 403237.6811981201, ROI : 80.65, Period : 9, Multiplier : 2.0, TotalTrades : 1917
ADANIPORTS.NS :	 Earning from investing 10000 is $1269.05 	 (ROI = 12.69%) 	 Trades=30
ASIANPAINT.NS :	 Earning from investing 10000 is $12537.7 	 (ROI = 125.38%) 	 Trades=27
AXISBANK.NS :	 Earning from investing 10000 is $-5419.9 	 (ROI = -54.2%) 	 Trades=34
BAJAJ-AUTO.NS :	 Earning from investing 10000 is $-232.7 	 (ROI = -2.33%) 	 Trades=28
BAJFINANCE.NS :	 Earning from investing 10000 is $95874.24 	 (ROI = 958.74%) 	 Trades=27
BAJAJFINSV.NS :	 Earning from investing 10000 is $76307.49 	 (ROI = 763.07%) 	 Trades=25
BHARTIARTL.NS :	 Earning from investing 10000 is $4158.01 	 (ROI = 41.58%) 	 Trades=27
BPCL.NS :	 Earning from investing 10000 is $-1301.47 	 (ROI = -13.01%) 	 Trades=33
BRITANNIA.NS :	 Earning from investing 10000 is $14914.05 	 (ROI = 149.14%) 	 Trades=24
CIPLA.NS :	 Earning fro

RELIANCE.NS :	 Earning from investing 10000 is $12011.08 	 (ROI = 120.11%) 	 Trades=19
SBIN.NS :	 Earning from investing 10000 is $-188.9 	 (ROI = -1.89%) 	 Trades=18
SBILIFE.NS :	 Earning from investing 10000 is $-891.85 	 (ROI = -8.92%) 	 Trades=9
SHREECEM.NS :	 Earning from investing 10000 is $6792.05 	 (ROI = 67.92%) 	 Trades=13
SUNPHARMA.NS :	 Earning from investing 10000 is $-1872.55 	 (ROI = -18.73%) 	 Trades=15
TATAMOTORS.NS :	 Earning from investing 10000 is $19024.81 	 (ROI = 190.25%) 	 Trades=15
TATASTEEL.NS :	 Earning from investing 10000 is $15432.36 	 (ROI = 154.32%) 	 Trades=14
TCS.NS :	 Earning from investing 10000 is $-256.33 	 (ROI = -2.56%) 	 Trades=18
TATACONSUM.NS :	 Earning from investing 10000 is $43015.89 	 (ROI = 430.16%) 	 Trades=9
TECHM.NS :	 Earning from investing 10000 is $22886.29 	 (ROI = 228.86%) 	 Trades=11
TITAN.NS :	 Earning from investing 10000 is $24367.95 	 (ROI = 243.68%) 	 Trades=12
ULTRACEMCO.NS :	 Earning from investing 10000 is $4499.7 	 (ROI 

ICICIBANK.NS :	 Earning from investing 10000 is $-1741.37 	 (ROI = -17.41%) 	 Trades=65
INDUSINDBK.NS :	 Earning from investing 10000 is $554.2 	 (ROI = 5.54%) 	 Trades=61
INFY.NS :	 Earning from investing 10000 is $-6727.99 	 (ROI = -67.28%) 	 Trades=69
IOC.NS :	 Earning from investing 10000 is $12345.3 	 (ROI = 123.45%) 	 Trades=49
ITC.NS :	 Earning from investing 10000 is $-6523.47 	 (ROI = -65.23%) 	 Trades=60
JSWSTEEL.NS :	 Earning from investing 10000 is $6145.48 	 (ROI = 61.45%) 	 Trades=54
KOTAKBANK.NS :	 Earning from investing 10000 is $2888.53 	 (ROI = 28.89%) 	 Trades=55
LT.NS :	 Earning from investing 10000 is $-1406.17 	 (ROI = -14.06%) 	 Trades=58
M&M.NS :	 Earning from investing 10000 is $-4613.35 	 (ROI = -46.13%) 	 Trades=63
MARUTI.NS :	 Earning from investing 10000 is $-1564.15 	 (ROI = -15.64%) 	 Trades=65
NESTLEIND.NS :	 Earning from investing 10000 is $-1849.15 	 (ROI = -18.49%) 	 Trades=52
NTPC.NS :	 Earning from investing 10000 is $-4048.24 	 (ROI = -40.48%) 	 Tr

CIPLA.NS :	 Earning from investing 10000 is $-40.7 	 (ROI = -0.41%) 	 Trades=20
COALINDIA.NS :	 Earning from investing 10000 is $-6359.7 	 (ROI = -63.6%) 	 Trades=27
DIVISLAB.NS :	 Earning from investing 10000 is $14671.55 	 (ROI = 146.72%) 	 Trades=20
DRREDDY.NS :	 Earning from investing 10000 is $-1011.3 	 (ROI = -10.11%) 	 Trades=26
EICHERMOT.NS :	 Earning from investing 10000 is $-2462.69 	 (ROI = -24.63%) 	 Trades=27
GRASIM.NS :	 Earning from investing 10000 is $17492.63 	 (ROI = 174.93%) 	 Trades=19
HCLTECH.NS :	 Earning from investing 10000 is $3392.89 	 (ROI = 33.93%) 	 Trades=22
HDFC.NS :	 Earning from investing 10000 is $13882.7 	 (ROI = 138.83%) 	 Trades=17
HDFCBANK.NS :	 Earning from investing 10000 is $9707.92 	 (ROI = 97.08%) 	 Trades=18
HDFCLIFE.NS :	 Earning from investing 10000 is $3904.3 	 (ROI = 39.04%) 	 Trades=12
HEROMOTOCO.NS :	 Earning from investing 10000 is $-4107.1 	 (ROI = -41.07%) 	 Trades=25
HINDALCO.NS :	 Earning from investing 10000 is $13762.35 	 (ROI = 

UPL.NS :	 Earning from investing 10000 is $9990.12 	 (ROI = 99.9%) 	 Trades=9
WIPRO.NS :	 Earning from investing 10000 is $26478.98 	 (ROI = 264.79%) 	 Trades=7
Investments : 500000, Returns : 570891.9911575317, ROI : 114.18, Period : 10, Multiplier : 5, TotalTrades : 481
Best parameter set: ATR Period=2, Multiplier=5, ROI=126.53


# BackTest ST for given ATR

In [ ]:
atr_period = 10
atr_multiplier = 2.75
returns, investment= STBackTest(nseDF,atr_period,atr_multiplier)
roi = round(returns/investment*100,2)
print(f'Investments : {investment}, Returns : {returns}, ROI : {roi}')

In [ ]:

# input variables

atr_period=4
atr_multiplier=4

symbol = 'ADANIPORTS.NS'
df = yf.download(symbol, start='2019-01-01')

#generate Supertrend for each ticker
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)

ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []
Earning = []
entry, exit, earning, roi, tradeCount = backtest_supertrend(df, symbol, 10000)
Earning.append(earning)
ROI.append(roi)
investment.append(10000)
totalTrades.append(tradeCount)
tickerList.append(ticker)
entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
result['Symbol'] = ticker = "SBIN"
result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)

print(result)

print(f'ROI = {roi}, TradeCount : {tradeCount}')



In [ ]:
entry, exit, roi = backtest_supertrend(df, 10000)

In [ ]:
# find_optimal_parameter for single stock
df = yf.download('TSLA', start='2010-01-01')
optimal_param = find_optimal_parameter(df, 'TSLA')
print(f'Best parameter set: ATR Period={optimal_param[0]}, Multiplier={optimal_param[1]}, ROI={optimal_param[2]}')


In [ ]:
#Nifty50 + Nift N50 + Nifty MidCap50 - Petroleum
tickerList = ['BAJFINANCE.NS', 'MARUTI.NS', 'HINDALCO.NS', 'SBIN.NS', 'BAJAJFINSV.NS', 'BRITANNIA.NS', 'ASIANPAINT.NS', 'ICICIBANK.NS', 'BHARTIARTL.NS', 'TATAMOTORS.NS', 'SUNPHARMA.NS', 'INFY.NS', 'ADANIPORTS.NS', 'TATASTEEL.NS', 'GRASIM.NS', 'JSWSTEEL.NS', 'HDFCBANK.NS', 'INDUSINDBK.NS', 'HCLTECH.NS', 'TECHM.NS', 'ULTRACEMCO.NS', 'M&M.NS', 'RELIANCE.NS', 'WIPRO.NS', 'LT.NS', 'SHREECEM.NS', 'TATACONSUM.NS', 'HDFC.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'HEROMOTOCO.NS', 'TITAN.NS', 'EICHERMOT.NS', 'DRREDDY.NS', 'TCS.NS', 'CIPLA.NS', 'SBILIFE.NS', 'HINDUNILVR.NS', 'NESTLEIND.NS', 'UPL.NS', 'DIVISLAB.NS', 'KOTAKBANK.NS', 'HDFCLIFE.NS', 'BIOCON.NS', 'AUROPHARMA.NS', 'JINDALSTEL.NS', 'DMART.NS', 'CADILAHC.NS', 'LUPIN.NS', 'BANKBARODA.NS', 'BAJAJHLDNG.NS', 'ACC.NS', 'ADANIENT.NS', 'PNB.NS', 'BOSCHLTD.NS', 'MCDOWELL-N.NS', 'SAIL.NS', 'HDFCAMC.NS', 'ICICIGI.NS', 'TORNTPHARM.NS', 'INDIGO.NS', 'PEL.NS', 'SBICARD.NS', 'BERGEPAINT.NS', 'MARICO.NS', 'BANDHANBNK.NS', 'DLF.NS', 'LTI.NS', 'INDUSTOWER.NS', 'AMBUJACEM.NS', 'PIIND.NS', 'NMDC.NS', 'COLPAL.NS', 'YESBANK.NS', 'JUBLFOOD.NS', 'CHOLAFIN.NS', 'PIDILITIND.NS', 'NAUKRI.NS', 'SIEMENS.NS', 'MUTHOOTFIN.NS', 'GODREJCP.NS', 'DABUR.NS', 'APOLLOHOSP.NS', 'HAVELLS.NS', 'ADANIGREEN.NS', 'ICICIPRULI.NS', 'IGL.NS', 'VEDL.NS', 'ADANITRANS.NS', 'GLAND.NS', 'PGHH.NS', 'MINDTREE.NS', 'IRCTC.NS', 'BALKRISIND.NS', 'BHARATFORG.NS', 'BATAINDIA.NS', 'AUBANK.NS', 'APOLLOTYRE.NS', 'MPHASIS.NS', 'ASHOKLEY.NS', 'TRENT.NS', 'PAGEIND.NS', 'SUNTV.NS', 'AARTIIND.NS', 'LTTS.NS', 'CANBK.NS', 'M&MFIN.NS', 'GLENMARK.NS', 'TVSMOTOR.NS', 'SRTRANSFIN.NS', 'RAMCOCEM.NS', 'EXIDEIND.NS', 'SRF.NS', 'IDFCFIRSTB.NS', 'GODREJPROP.NS', 'ZEEL.NS', 'UBL.NS', 'BANKINDIA.NS', 'LICHSGFIN.NS', 'MRF.NS', 'RECLTD.NS', 'TATAPOWER.NS', 'MANAPPURAM.NS', 'L&TFH.NS', 'ESCORTS.NS', 'ALKEM.NS', 'ASTRAL.NS', 'COFORGE.NS', 'MFSL.NS', 'HAL.NS', 'IDEA.NS', 'FEDERALBNK.NS', 'GUJGASLTD.NS', 'PFC.NS', 'CUMMINSIND.NS', 'VOLTAS.NS', 'BEL.NS', 'TORNTPOWER.NS', 'CONCOR.NS']

#BlueChip Trend from TV is giving more returns
#tickerList=['IOB.NS', 'NHPC.NS', 'UNIONBANK.NS', 'IDBI.NS', 'PNB.NS', 'IDFCFIRSTB.NS', 'BHEL.NS', 'BANKINDIA.NS', 'BANKBARODA.NS', 'L&TFH.NS', 'ADANIPOWER.NS', 'TATAPOWER.NS', 'NTPC.NS', 'IOC.NS', 'ONGC.NS', 'SAIL.NS', 'ABCAPITAL.NS', 'ASHOKLEY.NS', 'COALINDIA.NS', 'CANBK.NS', 'TATAMTRDVR.NS', 'GAIL.NS', 'NMDC.NS', 'MOTHERSUMI.NS', 'VEDL.NS', 'HINDPETRO.NS', 'TATAMOTORS.NS', 'AMBUJACEM.NS', 'HINDZINC.NS', 'HINDALCO.NS', 'JINDALSTEL.NS', 'CROMPTON.NS', 'SBIN.NS', 'TORNTPOWER.NS', 'LICHSGFIN.NS', 'MARICO.NS', 'BPCL.NS', 'GUJGASLTD.NS', 'IGL.NS', 'LAURUSLABS.NS', 'BHARTIARTL.NS', 'DABUR.NS', 'ICICIPRULI.NS', 'WIPRO.NS', 'CHOLAFIN.NS', 'SUNTV.NS', 'MCDOWELL-N.NS', 'CADILAHC.NS', 'TATACONSUM.NS', 'CONCOR.NS', 'ICICIBANK.NS', 'TVSMOTOR.NS', 'HDFCLIFE.NS', 'BHARATFORG.NS', 'JSWSTEEL.NS', 'SUNPHARMA.NS', 'AXISBANK.NS', 'ADANIPORTS.NS', 'CUMMINSIND.NS', 'BERGEPAINT.NS', 'UPL.NS', 'M&M.NS', 'TRENT.NS', 'GODREJCP.NS', 'CIPLA.NS', 'RAMCOCEM.NS', 'HCLTECH.NS', 'SBILIFE.NS', 'AUBANK.NS', 'INDUSINDBK.NS', 'VOLTAS.NS', 'HAVELLS.NS', 'AUROPHARMA.NS', 'SBICARD.NS', 'TECHM.NS', 'TATASTEEL.NS', 'LUPIN.NS', 'MUTHOOTFIN.NS', 'ADANIGREEN.NS', 'ADANIENT.NS', 'GODREJPROP.NS', 'ATGL.NS', 'INFY.NS', 'GRASIM.NS', 'ADANITRANS.NS', 'SRTRANSFIN.NS', 'HDFCBANK.NS', 'LT.NS', 'TITAN.NS', 'AARTIIND.NS', 'PEL.NS', 'KOTAKBANK.NS', 'DEEPAKNTR.NS', 'IRCTC.NS', 'ACC.NS', 'PIDILITIND.NS', 'HINDUNILVR.NS', 'MINDTREE.NS', 'HDFC.NS', 'EICHERMOT.NS', 'ASIANPAINT.NS']

result=[] 
for symbol in tickerList: 
    data = []
    df = yf.download(symbol, start='2019-01-01') 
    data.append(symbol)
    data.append(df)
    result.append(data)
   

In [ ]:
returns, investment= STBackTest(result,10, 2.5)
roi = round(returns/investment*100,2)
roi

# 1. Get all Trades for given input
# 2. Generate Calls


In [14]:
#Get all Trades for given input
# 1. Download Data



tickerList = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1].Symbol.to_list()

startDate='2015-01-01'
endDate='2019-12-31'

#BlueChip Trend from TV is giving more returns
#tickerList=['IOB', 'NHPC', 'UNIONBANK', 'IDBI', 'PNB', 'IDFCFIRSTB', 'BHEL', 'BANKINDIA', 'BANKBARODA', 'L&TFH', 'ADANIPOWER', 'TATAPOWER', 'NTPC', 'IOC', 'ONGC', 'SAIL', 'ABCAPITAL', 'ASHOKLEY', 'COALINDIA', 'CANBK', 'TATAMTRDVR', 'GAIL', 'NMDC', 'MOTHERSUMI', 'VEDL', 'HINDPETRO', 'TATAMOTORS', 'AMBUJACEM', 'HINDZINC', 'HINDALCO', 'JINDALSTEL', 'CROMPTON', 'SBIN', 'TORNTPOWER', 'LICHSGFIN', 'MARICO', 'BPCL', 'GUJGASLTD', 'IGL', 'LAURUSLABS', 'BHARTIARTL', 'DABUR', 'ICICIPRULI', 'WIPRO', 'CHOLAFIN', 'SUNTV', 'MCDOWELL-N', 'CADILAHC', 'TATACONSUM', 'CONCOR', 'ICICIBANK', 'TVSMOTOR', 'HDFCLIFE', 'BHARATFORG', 'JSWSTEEL', 'SUNPHARMA', 'AXISBANK', 'ADANIPORTS', 'CUMMINSIND', 'BERGEPAINT', 'UPL', 'M&M', 'TRENT', 'GODREJCP', 'CIPLA', 'RAMCOCEM', 'HCLTECH', 'SBILIFE', 'AUBANK', 'INDUSINDBK', 'VOLTAS', 'HAVELLS', 'AUROPHARMA', 'SBICARD', 'TECHM', 'TATASTEEL', 'LUPIN', 'MUTHOOTFIN', 'ADANIGREEN', 'ADANIENT', 'GODREJPROP', 'ATGL', 'INFY', 'GRASIM', 'ADANITRANS', 'SRTRANSFIN', 'HDFCBANK', 'LT', 'TITAN', 'AARTIIND', 'PEL', 'KOTAKBANK', 'DEEPAKNTR', 'IRCTC', 'ACC', 'PIDILITIND', 'HINDUNILVR', 'MINDTREE', 'HDFC', 'EICHERMOT', 'ASIANPAINT']
nseDF = NSEData(tickerList,startDate,endDate)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [17]:
# 2. Generate Calls

#Nifty50 BackTest
# 4,5 : Investment : 500000; FinalAmount : 1070460.0; Gains : 114.092; TotalTraded : 529
# 4,4 : Investment : 500000; FinalAmount : 1088459.0; Gains : 117.69179999999999; TotalTraded : 737
# 9,5 : ROI : 119.03 Inv : 500000 Return : 595132.8564 Trdaes : 484



atr_period=9
atr_multiplier=5


ROI =[]
tickerList =[]
investment = []
totalTrades = []
finalResults = []
Earning = []

for data in nseDF:
    df=data[1]
    ticker=data[0]
    supertrend = Supertrend(df, atr_period, atr_multiplier)
    df = df.join(supertrend)
    entry, exit, earning, roi, tradeCount = backtest_supertrend(df, ticker, 10000)
    Earning.append(earning)
    ROI.append(roi)
    investment.append(10000)
    totalTrades.append(tradeCount)
    tickerList.append(ticker)
    entryDF = pd.DataFrame(entry,columns=['Day','BuyPrice','BuyDate'])
    exitDF = pd.DataFrame(exit,columns=['Day','SellPrice','SellDate','Equity'])
    result = pd.merge(entryDF, exitDF, left_index=True, right_index=True)
    result['PnL_Percent'] = (result['SellPrice'] - result['BuyPrice'])/result['BuyPrice']*100
    result['Symbol'] = ticker
    result.drop(['Day_x', 'Day_y'], axis = 1,inplace=True)
    finalResults.append(result)
    

    
resultDF = pd.DataFrame(
    {'Ticker': tickerList,
     'Investment': investment,
     'FinalReturns': ROI,
     'TotalTrades': totalTrades
    })
resultDF['FinalAmount'] = resultDF['Investment'] + resultDF['Investment']*(resultDF['FinalReturns']/100)

finalResults = pd.concat(finalResults)
finalResults
finalResults = finalResults[['Symbol','BuyDate', 'BuyPrice', 'SellDate', 'SellPrice','PnL_Percent','Equity']]
finalResults.to_csv('result/SuperTrend_BackTest_Trades.csv')
resultDF.to_csv('result/SuperTrend_Final_Result.csv')
totalInvestment = resultDF['Investment'].sum()
totalFinalAmount = resultDF['FinalAmount'].sum()
gains = ((totalFinalAmount - totalInvestment)/totalInvestment)*100
totalFinalTrades = resultDF['TotalTrades'].sum()


print(f'Investment : {totalInvestment}; FinalAmount : {totalFinalAmount}; Gains : {gains}; TotalTraded : {totalFinalTrades}')

ADANIPORTS.NS :	 Earning from investing 10000 is $-3186.85 	 (ROI = -31.87%) 	 Trades=8
ASIANPAINT.NS :	 Earning from investing 10000 is $5464.85 	 (ROI = 54.65%) 	 Trades=7
AXISBANK.NS :	 Earning from investing 10000 is $2386.5 	 (ROI = 23.87%) 	 Trades=7
BAJAJ-AUTO.NS :	 Earning from investing 10000 is $-2647.8 	 (ROI = -26.48%) 	 Trades=9
BAJFINANCE.NS :	 Earning from investing 10000 is $62583.05 	 (ROI = 625.83%) 	 Trades=6
BAJAJFINSV.NS :	 Earning from investing 10000 is $31097.2 	 (ROI = 310.97%) 	 Trades=6
BHARTIARTL.NS :	 Earning from investing 10000 is $2891.89 	 (ROI = 28.92%) 	 Trades=6
BPCL.NS :	 Earning from investing 10000 is $4442.31 	 (ROI = 44.42%) 	 Trades=8
BRITANNIA.NS :	 Earning from investing 10000 is $14716.08 	 (ROI = 147.16%) 	 Trades=7
CIPLA.NS :	 Earning from investing 10000 is $-2969.65 	 (ROI = -29.7%) 	 Trades=7
COALINDIA.NS :	 Earning from investing 10000 is $-3732.05 	 (ROI = -37.32%) 	 Trades=8
DIVISLAB.NS :	 Earning from investing 10000 is $11493.73 	 

In [1]:
finalResults

NameError: name 'finalResults' is not defined